# Realizar pruebas con modelos de embeddings de sentence-transformer

La idea es realizar pruebas en donde se guarde info en el espacio vectorial y realizar busquedas de similitud con un input

-------------------------------------

In [21]:
import warnings
warnings.filterwarnings('ignore')
from langchain_community.document_loaders import PyPDFLoader
import os
import re
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents.base import Document
import pandas as pd
pd.options.display.max_columns = None
pd.options.display.max_rows = None
from sentence_transformers import SentenceTransformer, util

## Load file

In [2]:
def load_files(path_files=str, file=None, debug=None):
    """
    Cargador de documentos en formato PDF. Ademas, se realiza una limpieza para eliminar saltos de linea y múltiples espacios en el texto,
    esto se hace pensando en que los fragmentos y embeddings sean mas coherentes y limpios.

    Args:
        path_files: Ruta en donde se encuentran los archivos.
        file: Si se desea cargar un unico archivo de la ruta
        debug: Booleano, si se quiere imprimir información del objeto

    Returns:
        Una lista de objetos Document.
    """

    docs = os.listdir(path_files)
    docs = [path_files + x for x in docs]
    if file != None:
        docs = [x for x in docs if file in x]

    documents = []
    for i in docs:
        loader = PyPDFLoader(i)
        data = loader.load()
        documents.extend(data)

    clean_documents = []
    for j in documents:
        clean = re.sub(r'\s+', ' ', j.page_content)
        clean = Document(page_content=clean, metadata=j.metadata)
        clean_documents.append(clean)

    if debug == True:
        print(f'El objeto retornado por la función es de tipo: {type(clean_documents)}. Y cada subobjeto es de tipo: {type(clean_documents[0])}')
        print(f'En total se cargaron {len(clean_documents)} paginas')
        
    return clean_documents

In [3]:
path = "G:/Mi unidad/Universidad/Pregrado/1,2,3,4/Estadistica/"
pages = load_files(path_files=path, file='HistoriaEstadistica', debug=True)

El objeto retornado por la función es de tipo: <class 'list'>. Y cada subobjeto es de tipo: <class 'langchain_core.documents.base.Document'>
En total se cargaron 15 paginas


In [4]:
pages[:10]

[Document(page_content='DESARROLLO HISTÓRICO DE LA ESTADÍSTICA Extractado por Jorge Galbiati Riesco Las fuentes de la Estadística la constituyen los censos y recuentos, los juegos de azar, la inferencia inductiva basada en datos empíricos, y el tratamiento de los errores en las mediciones. Se sabe que Cesar Augusto decretó que todo el imperio fuera sometido al pago de impuestos, para lo cual previamente debería conducirse un censo de las personas. Mil años después, Guillermo el Conquistador ordenó que se hiciera un registro de todos los bienes que hubieran en Inglaterra, para fines tributarios y militares, el llamado "Domesday Book". Una aplicación de la probab ilidad empírica a los seguros de buques se encuentra en Flandes, en el siglo XIV. La teoría a de la probabilidad es una disciplina matemática que fundamenta la Estadística como una lógica y una metodologí a para la medición y el estudio de la incertidumbre, en la planeación e interpretación de la observación y la experimentación

---------------------------------------

## Fragments

In [5]:
def fragments(obj=list, chunk_size=int, chunk_overlap=None, debug=None):
    """
    Fragmentar los documentos previamente cargados.

    Args:
        obj: Lista con objetos tipo documents. En otras palabras tiene que ser una lista con las paginas de los documentos cargados.
        chunk_size: Tamaño de los fragmentos o numero de caracteres de los fragmentos.
        chunk_overlap: Superposicion de los fragmentos. Se recomienda que sea entre el 10% y 15% del chunk_size. Por defecto 10%
        debug: Booleano, si se quiere imprimir información del objeto

    Returns:
        Objeto document fragmentado.
    """

    if chunk_overlap == None:
        chunk_overlap = chunk_size*0.15

    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    texts = splitter.split_documents(obj)

    if debug == True:
        print(f'Los fragmentos son de tamaño: {chunk_size} y la superposicion es igual a: {chunk_overlap}')
        print(f'El número de fragmentos total es igual a: {len(texts)}')

    return texts

In [6]:
pages = fragments(pages, 500, 0.10, debug=True)

Los fragmentos son de tamaño: 500 y la superposicion es igual a: 0.1
El número de fragmentos total es igual a: 79


In [7]:
pages[:10]

[Document(page_content='DESARROLLO HISTÓRICO DE LA ESTADÍSTICA Extractado por Jorge Galbiati Riesco Las fuentes de la Estadística la constituyen los censos y recuentos, los juegos de azar, la inferencia inductiva basada en datos empíricos, y el tratamiento de los errores en las mediciones. Se sabe que Cesar Augusto decretó que todo el imperio fuera sometido al pago de impuestos, para lo cual previamente debería conducirse un censo de las personas. Mil años después, Guillermo el Conquistador ordenó que se hiciera un', metadata={'source': 'G:/Mi unidad/Universidad/Pregrado/1,2,3,4/Estadistica/HistoriaEstadistica.pdf', 'page': 0}),
 Document(page_content='registro de todos los bienes que hubieran en Inglaterra, para fines tributarios y militares, el llamado "Domesday Book". Una aplicación de la probab ilidad empírica a los seguros de buques se encuentra en Flandes, en el siglo XIV. La teoría a de la probabilidad es una disciplina matemática que fundamenta la Estadística como una lógica y 

---------------------------------------

## Create a dataframe

In [8]:
register = []
for i in pages:
    register.append(i.page_content)
df = pd.DataFrame({'script':register})
df['length_scripts'] = df['script'].apply(lambda x: len(x))
print(df.shape)
df.sample(5)

(79, 2)


,script,length_scripts
42,aleatoria es la base de una teoría científica ...,496
5,"aleatoria, John Graunt es considerado por algu...",498
38,"estudio sobre una población, es",31
47,periodo se establecen las condiciones bajo las...,498
17,fue el tema dominante de los estadísticos del ...,499


---------------------------------------------

## Embeddings model

In [9]:
model = SentenceTransformer('sentence-transformers/multi-qa-mpnet-base-dot-v1')
model.get_buffer

You try to use a model that was created with version 3.0.0.dev0, however, your version is 2.7.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





<bound method Module.get_buffer of SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)>

-------------------------------------------

## Save embeddings in df

In [10]:
embeddings = model.encode(df['script'], show_progress_bar=True)

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

In [11]:
embeddings.shape

(79, 768)

In [12]:
df['embeddings'] = embeddings.tolist()

In [13]:
df.sample(10)

,script,length_scripts,embeddings
48,"o r l a E s t a d í s t i c a , p e r o trabaj...",492,"[-0.36984357237815857, -0.1156497672200203, -0..."
74,estadísticos se vuelven más complejos. Los mét...,500,"[-0.6187661290168762, 0.05496549606323242, -0...."
77,"Mundo de las Matemáticas"", ed. Grjaldo S.A., 1...",50,"[-0.14384321868419647, 0.021477237343788147, -..."
16,"En 1805, el matemático francés Adrien Marie Le...",498,"[-0.43346428871154785, 0.1469012200832367, -0...."
53,"superficies de respuesta, un procedimiento fin...",497,"[-0.20427772402763367, -0.128569558262825, -0...."
22,fenómenos pueden ser representados probab ilís...,491,"[-0.3293474316596985, -0.09594462066888809, -0..."
3,Girolamo Cardano (1501- 1576) y Galileo Galile...,324,"[-0.23017717897891998, 0.28068116307258606, -0..."
76,"Bibliografía T.W. Anderson: ""An introduction t...",497,"[-0.26468056440353394, 0.05537289381027222, -0..."
62,"distribución pertenece a alguna familia, cuya ...",498,"[-0.5775436758995056, 0.2799033522605896, -0.1..."
27,"A partir de 1880, tres hombres, Francis Galton...",495,"[-0.1696656346321106, 0.3837101459503174, -0.2..."


--------------------------------------

## Searching

In [ ]:
def search_similarity(query=str, dataframe=pd.DataFrame, num_return=None):
    """Función para realizar busquedas de coincidencia entre una frase o palabra en un dataframe en donde se encuentran frases con su 
    respectiva representación vectorial
    
    Arg:
        query: Frase o palabra que se quiere buscar
        dataframe: Dataframe en donde se encuentran las frases junto con su representación vectorial
        num_return: Numero de frases similares que se quieren retornar

    Returns:
        Lista con las frases que mas coinciden con la frase o palabra ingresada
    """

    if num_return == None:
        num_return = 3

    query_embedding = model.encode([query])
    dataframe['similarity'] = dataframe.embeddings.apply(lambda x : util.cos_sim(x,query_embedding[0]))
    dataframe = dataframe.sort_values('similarity',ascending=False).head(num_return)

    print(dataframe['script'])

In [14]:
query_embedding = model.encode(['Estadística Bayesiana'])

df['similarity'] = df.embeddings.apply(lambda x : util.cos_sim(x,query_embedding[0]))

In [15]:
df.sample(5)

,script,length_scripts,embeddings,similarity
36,una de sus aplicaciones es el probar el ajuste...,499,"[-0.2803966999053955, -0.2062561959028244, -0....",[[tensor(0.4738)]]
30,forma conjunta . Esta ya había sido conocida p...,493,"[-0.5089791417121887, -0.010379701852798462, -...",[[tensor(0.5014)]]
46,"muestreo estratificado , que asume que existen...",494,"[-0.44724780321121216, 0.04111926257610321, -0...",[[tensor(0.3790)]]
73,los medios existentes hasta entonces. Las difi...,117,"[-0.3203105330467224, 0.028431817889213562, -0...",[[tensor(0.4439)]]
6,"y medicina. En su obra ""Ars Conjectandi"", intr...",499,"[-0.3316954970359802, -0.33490312099456787, -0...",[[tensor(0.3358)]]


In [17]:
df = df.sort_values('similarity',ascending=False).head(5)

In [24]:
print(df['script'])

11    temprano de la Estadística. Irónicamente, sus ...
14    Causas de Eventos", de 1774, y "Memoria sobre ...
66    diseño y análisis de encuestas y tests. Abraha...
67    característica de los Estados Unidos, define l...
39    a n t i g u a . E n e s t a i d e a s e f u n ...
Name: script, dtype: object
